# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [80]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import gmaps.datasets
import os
from ipywidgets.embed import embed_minimal_html

# Import API key
from api_keys import g_key

gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [81]:
fpath = '../output_data/NathanAsh_cities.csv'

city = pd.read_csv(fpath)
city

,Unnamed: 0,City,Latitude,Longitude,Max Tempurature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country
0,1,hermanus,-34.42,19.23,55.004,87.0,99,3.58,ZA
1,3,saint george,37.10,-113.58,102.200,13.0,1,4.60,US
2,7,port macquarie,-31.43,152.92,71.996,35.0,0,2.24,AU
3,9,najran,17.49,44.13,84.200,25.0,20,2.10,SA
4,11,piacabucu,-10.41,-36.43,71.798,86.0,0,1.62,BR
...,...,...,...,...,...,...,...,...,...
515,609,panagar,23.30,79.98,75.200,94.0,90,3.10,IN
516,610,waipawa,-41.41,175.52,51.008,61.0,0,2.60,NZ
517,611,zhangjiakou,40.81,114.88,78.278,55.0,98,3.93,CN
518,612,tabou,4.42,-7.35,76.550,81.0,100,4.24,CI


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [82]:


loc = city[['Latitude','Longitude']]
hum = city['Humidity (%)'].astype(float)


In [83]:

fig = gmaps.figure()
heatH = gmaps.heatmap_layer(loc,weights = hum, 
                           dissipating = False, max_intensity = 10,
                           point_radius = .5)
fig.add_layer(heatH)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [84]:
ideal = city.copy()
# ideal['Humidity']  = pd.to_numeric(ideal['Humidity'])

# for index,row in city_df.iterrows():
ideal = ideal.loc[ideal['Max Tempurature (F)'].gt(80) == False]
ideal = ideal.loc[ideal['Max Tempurature (F)'].gt(70) == True]
ideal = ideal.loc[ideal['Wind Speed (mph)'].gt(10) == False]
ideal = ideal.loc[ideal['Cloudiness (%)'].gt(0) == False]
ideal = ideal.dropna(how = 'any')
ideal = ideal.drop(['Unnamed: 0'], axis =1)
ideal


,City,Latitude,Longitude,Max Tempurature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country
2,port macquarie,-31.43,152.92,71.996,35.0,0,2.24,AU
4,piacabucu,-10.41,-36.43,71.798,86.0,0,1.62,BR
19,carnarvon,-24.87,113.63,71.600,53.0,0,6.20,AU
52,airai,-8.93,125.41,77.198,30.0,0,2.74,TL
75,marsa matruh,31.35,27.25,77.000,78.0,0,2.60,EG
91,madrid,40.42,-3.70,71.006,48.0,0,2.60,ES
100,sao miguel do araguaia,-13.28,-50.16,75.038,39.0,0,1.93,BR
172,abu samrah,35.30,37.18,70.178,62.0,0,3.51,SY
178,jalu,29.03,21.55,76.514,44.0,0,4.02,LY
204,sayat,38.78,63.88,77.000,47.0,0,2.10,TM


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [85]:
#For loop 

params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}


ideal["Hotel Name"] = ""

for index, row in ideal.iterrows():
    lat = row['Latitude']
    lng = row['Longitude']

    params['location'] = f"{lat},{lng}"
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    urlpar = requests.get(url,  params=params)
    urlpar = urlpar.json()
    try:
        ideal.loc[index, 'Hotel Name'] = urlpar['results'][0]['name']
        print(f'Hotel in {row[0]} found')
        
    except:
        print(f'Hotel in {row[0]} not found')
        
ideal
    


Hotel in port macquarie found
Hotel in piacabucu found
Hotel in carnarvon found
Hotel in airai found
Hotel in marsa matruh found
Hotel in madrid found
Hotel in sao miguel do araguaia found
Hotel in abu samrah not found
Hotel in jalu found
Hotel in sayat not found
Hotel in prado found
Hotel in zabol not found
Hotel in awjilah not found
Hotel in gat found
Hotel in aparecida do taboado found
Hotel in guerrero negro found
Hotel in karatau found
Hotel in nouadhibou found
Hotel in kosonsoy found
Hotel in valdepenas found
Hotel in sabha found
Hotel in nalut not found
Hotel in chardara found


,City,Latitude,Longitude,Max Tempurature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Hotel Name
2,port macquarie,-31.43,152.92,71.996,35.0,0,2.24,AU,Rydges Port Macquarie
4,piacabucu,-10.41,-36.43,71.798,86.0,0,1.62,BR,Piaçabuçu News
19,carnarvon,-24.87,113.63,71.600,53.0,0,6.20,AU,Hospitality Carnarvon
52,airai,-8.93,125.41,77.198,30.0,0,2.74,TL,Juvinal Martin
75,marsa matruh,31.35,27.25,77.000,78.0,0,2.60,EG,Beau Site
91,madrid,40.42,-3.70,71.006,48.0,0,2.60,ES,Iberostar Las Letras Gran Vía
100,sao miguel do araguaia,-13.28,-50.16,75.038,39.0,0,1.93,BR,Hotel Executivo Palace
172,abu samrah,35.30,37.18,70.178,62.0,0,3.51,SY,
178,jalu,29.03,21.55,76.514,44.0,0,4.02,LY,Jalu Hotel
204,sayat,38.78,63.88,77.000,47.0,0,2.10,TM,


In [86]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
hotel = ideal.copy()
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotelinfo = [info_box_template.format(**row) for index, row in hotel.iterrows()]
locations = hotel[["Latitude", "Longitude"]]
hotel

,City,Latitude,Longitude,Max Tempurature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Hotel Name
2,port macquarie,-31.43,152.92,71.996,35.0,0,2.24,AU,Rydges Port Macquarie
4,piacabucu,-10.41,-36.43,71.798,86.0,0,1.62,BR,Piaçabuçu News
19,carnarvon,-24.87,113.63,71.600,53.0,0,6.20,AU,Hospitality Carnarvon
52,airai,-8.93,125.41,77.198,30.0,0,2.74,TL,Juvinal Martin
75,marsa matruh,31.35,27.25,77.000,78.0,0,2.60,EG,Beau Site
91,madrid,40.42,-3.70,71.006,48.0,0,2.60,ES,Iberostar Las Letras Gran Vía
100,sao miguel do araguaia,-13.28,-50.16,75.038,39.0,0,1.93,BR,Hotel Executivo Palace
172,abu samrah,35.30,37.18,70.178,62.0,0,3.51,SY,
178,jalu,29.03,21.55,76.514,44.0,0,4.02,LY,Jalu Hotel
204,sayat,38.78,63.88,77.000,47.0,0,2.10,TM,


In [87]:
# Add marker layer ontop of heat map
# Display figure

marker = gmaps.marker_layer(locations, info_box_content = hotelinfo)
fig.add_layer(marker)
fig

Figure(layout=FigureLayout(height='420px'))